In [2]:
import requests
import os

os.environ["http_proxy"] = ""
os.environ["https_proxy"] = ""

DEFAULT_HOST_IP = "10.112.100.138"


def QnA(request, host_ip=DEFAULT_HOST_IP):
    url = f"http://{host_ip}:8888/v1/chatqna"

    headers = {"Content-Type": "application/json"}

    response = requests.post(url, headers=headers, json=request)
    return response


def upload_rag_file(rag_file, host_ip=DEFAULT_HOST_IP):
    url = f"http://{host_ip}:6007/v1/dataprep/ingest"

    with open(rag_file, "rb") as f:
        files = {"files": f}
        response = requests.post(url, files=files)

    return response

def delete_rag_file(rag_file, host_ip=DEFAULT_HOST_IP):
    url = f"http://{host_ip}:6007/v1/dataprep/delete"
    headers = {"Content-Type": "application/json"}
    payload = {"file_path": rag_file}

    response = requests.post(url, headers=headers, json=payload)
    return response


In [15]:
response=delete_rag_file("results.txt")
print("Status Code:", response.status_code)
print("Response Body:", response.text)

Status Code: 200
Response Body: {"status":true}


In [16]:
# download pdf file
# wget https://raw.githubusercontent.com/opea-project/GenAIComps/v1.1/comps/retrievers/redis/data/nke-10k-2023.pdf

# upload pdf file with dataprep
rag_file = "/home/sdp/lifeng/ai_for_validation/RAG_opea/results.txt"

upload_rag_file_response = upload_rag_file(rag_file)

print("Status Code:", upload_rag_file_response.status_code)
print("Response Body:", upload_rag_file_response.text)

Status Code: 200
Response Body: {"status":200,"message":"Data preparation succeeded"}


In [3]:
request = {
        "messages": "Unit test test_multiheadattention_fastpath_attn_mask_attn_mask_dim_2_key_padding_mask_dim_2_bool_xpu failed returned RuntimeError: output 1: meta disagrees with real impl ",
        "stream": False,
        "top_n": 3,       
    }

QnA_response=QnA(request)

print("Status Code:", QnA_response.status_code)
print("Response Body:", QnA_response.text)

if QnA_response.status_code==200:
    result=QnA_response.json()["choices"][0]["message"]["content"]
    answer = result.split("</think>")[-1].strip()
    print(answer)

Status Code: 200
Response Body: {"id":"chatcmpl-mmXwFUEzSpPWwDnJ4bq86e","object":"chat.completion","created":1744937735,"model":"chatqna","choices":[{"index":0,"message":{"role":"assistant","content":"Okay, so I'm trying to figure out why the unit test `test_multiheadattention_fastpath_attn_mask_attn_mask_dim_2_key_padding_mask_dim_2_bool_xpu` is failing with a RuntimeError about the output meta disagreeing with the real implementation. Let me go through the information provided step by step.\n\nFirst, I see that there are three search results provided. Each is a merged result with details about an issue. I need to check if the failed test is mentioned in any of these.\n\nLooking at the first result, issue 1432, the description mentions several failed tests, including the one I'm looking for: `test_multiheadattention_fastpath_attn_mask_attn_mask_dim_2_key_padding_mask_dim_2_bool_xpu`. The root cause is a bug in the scaled_dot_product_attention function due to a specific PyTorch commit,

In [ ]:
QnA_response.json()

In [4]:
result = result.replace("\\n\\n", "\n\n")
print(result)

Okay, so I'm trying to figure out why the unit test `test_multiheadattention_fastpath_attn_mask_attn_mask_dim_2_key_padding_mask_dim_2_bool_xpu` is failing with a RuntimeError about the output meta disagreeing with the real implementation. Let me go through the information provided step by step.

First, I see that there are three search results provided. Each is a merged result with details about an issue. I need to check if the failed test is mentioned in any of these.

Looking at the first result, issue 1432, the description mentions several failed tests, including the one I'm looking for: `test_multiheadattention_fastpath_attn_mask_attn_mask_dim_2_key_padding_mask_dim_2_bool_xpu`. The root cause is a bug in the scaled_dot_product_attention function due to a specific PyTorch commit, which caused incorrect output shapes and NaN outputs for fully masked rows. The assignee is LuFinch, and the state is open.

The second result, issue 1437, also mentions several test failures, but the tes